# Коллаборативная фильтрация

## Основная идея
Как понять, что пользователю $u$ нужно показать айтем $i$?
1. Если $u$ понравился айтем, похожий на $i$, то можно предположить, что ему понравится $i$.
2. Если ему не понравился айтем, похожий на $i$, ему, вероятно, не понравится и $i$.

Таким образом, идея состоит в том, чтобы посмотреть, насколько понравились $u$ айтемы, похожие на $i$.

## Item-to-item collaborative filtering

Реализуем на семинаре следующую вариацию item-to-item метода.
1. Пусть $U_i$ &mdash; множество пользователей, оценивших $i$. Определим, как мы будем определять похожести между айтемами. Определим похожести между айтемами, как
$$
         w(i, j) = \frac{\sum_{u\in U_i \cap U_j}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,j}-\bar{r}_u)^2}}.
$$
Обозначим за $S_k^{(i)}$ множество из $k$ наиболее близких к $i$ айтемов.
   
2. Пользователю $u$, оценившему множество айтемов $I_u$, будем рекомендовать $N$ наиболее подходящих ему айтемов. Для этого,
- В качестве кандидатов на попадание в топ возьмем айтемы $$C = \bigcup_{i \in I_u} S_k^{(i)} \setminus I_u.$$
- Для кандидатов $c \in C$ определим похожесть $c$ на историю пользователя $I_u$, как
\begin{equation}
 w(c,I_u) = \sum_{i\in I_u} w_{c,i}.
\end{equation}
- Вернем в качестве результата $N$ айтемов $c \in C$ с максимальной величиной $w(c, I_u)$.


### Import useful requirements

In [27]:
import math
import pathlib

import numpy as np
import polars as pl
import cupy
from cupy import sparse as cusparse
from scipy import sparse
import numba

In [28]:
%load_ext autoreload
%autoreload 2

from recs_utils.load_data import MovieLens100K
from recs_utils.matrix_ops import interactions_to_csc_matrix
from recs_utils.cuda import ADJUSTED_COSINE_SIM_BETWEEN_ITEMS, get_grid_thread_size

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Загрузка датасета

Для применения методов будем использовать датасет Movielens. Он представляет из себя оценки, которые пользователи поставили просмотренным фильмам и небольшие описания самих фильмов.

Для удобства изучения разных алгоритмов исследовательская группа, которая занимается разработкой датасета, подготовила данные разного объёма: 100к рейтингов, 1М, 10М, 20М. В этой работе мы будем пользоваться самым маленьким.

https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset

In [29]:
data_dir = pathlib.Path("data/movielens/ml-100k")

In [30]:
interactions = MovieLens100K.load_interactions(str(data_dir / "u.data"))
items = MovieLens100K.load_items(str(data_dir / "u.item"))

In [31]:
interactions.select(pl.col("rating").unique())

rating
f32
1.0
2.0
3.0
4.0
5.0


In [32]:
interactions.head()

user_id,item_id,rating
u32,u32,f32
1,1,5.0
1,2,3.0
1,3,4.0
1,4,3.0
1,5,3.0


In [33]:
len(interactions)

100000

In [34]:
with pl.Config() as config:
    config.set_fmt_str_lengths(100)
    print(items.head())

shape: (5, 2)
┌─────────┬───────────────────┐
│ item_id ┆ title             │
│ ---     ┆ ---               │
│ u32     ┆ str               │
╞═════════╪═══════════════════╡
│ 1       ┆ Toy Story (1995)  │
│ 2       ┆ GoldenEye (1995)  │
│ 3       ┆ Four Rooms (1995) │
│ 4       ┆ Get Shorty (1995) │
│ 5       ┆ Copycat (1995)    │
└─────────┴───────────────────┘


In [35]:
len(items)

1682

In [36]:
def get_direct_and_inv_mapping(interactions: pl.DataFrame, col_name: str):
    inv_mapping = dict(enumerate(interactions.select(
            pl.col(col_name).unique().sort()).to_series()))
    direct_mapping = {v: k for k, v in inv_mapping.items()}

    return direct_mapping, inv_mapping

### Предобработка датасета

In [37]:
# create the encoder
user_mapping, inv_user_mapping = get_direct_and_inv_mapping(interactions, "user_id")
item_mapping, inv_item_mapping = get_direct_and_inv_mapping(interactions, "item_id")

## Реализация алгоритма

### Шаг 1. Вычислить похожести между айтемами

#### adjustied cosine similarity

Вспомним, что каждый айтем можно представить как вектор оценок пользователей, тогда для оценки похожести можно использовать:
1. Косинусное расстояние
2. Ajusted Cosine Similarity
3. Евклидово расстояние
4. Манхэттенское расстояние
5. Коэффициент Жаккара



В item-based рекомендациях к-т adjustied cosine similarity доказал свою эффективность, поэтому будем использовать его. Схожесть между айтемами $i$ и $j$ считается по формуле:


\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}.
\end{equation}

Итак, чтобы вычислить похожесть между айтемами $i$ и $j$, нужно

1. Выявить всех пользователей, которые оценили оба айтема, т.е. $U_i \cap U_j$;
2. Нормализовать рейтинги айтемов $i$ и $j$;
3. Посчитать похожесть между векторами общих рейтингов $i$ и $j$.


Проведём подготовительную работу и нормализуем рейтинги всех пользователей:

In [38]:
def normalize(ratings: pl.DataFrame) -> pl.DataFrame:
    """
    Нормализует рейтинги по пользователям. Из каждого рейтинга вычитает средний рейтинг по пользователю.
    ratings: таблица рейтингов
    
    Возвращает: 
        Таблица, содержащая все колонки таблицы `ratings` и колонку `norm_rating` с нормализованными рейтингами.
    """
    return ratings.lazy().join(
        ratings.lazy().select(pl.col("user_id"), pl.col("rating")).groupby("user_id").agg(
        [
            pl.mean("rating").alias("mean_user_rating")
        ]
        ),
        on="user_id",
        how="inner").with_columns(
        (pl.col("rating") - pl.col("mean_user_rating")).alias("norm_rating")
        ).collect()

In [39]:
def test_normalize():
    test_df = pl.DataFrame({
        "user_id": [0, 0, 0, 1, 1],
        "item_id": [0, 1, 2, 1, 3],
        "rating": [2, 2, 5, 5, 5],
    })
    
    expected = pl.DataFrame({
        "user_id": [0, 0, 0, 1, 1],
        "item_id": [0, 1, 2, 1, 3],
        "rating": [2, 2, 5, 5, 5],
        "norm_rating": [-1, -1, 2, 0, 0]
    })    
    
    assert test_df.shape[0] == expected.shape[0], "Number of user-item interactions is different"
    assert test_df.shape[1] + 1 == expected.shape[1], "Number of columns is incorrect"
    assert normalize(test_df).select(pl.all().exclude("mean_user_rating")).frame_equal(expected), "Result is incorrect"
    
test_normalize()

In [40]:
norm_ratings = normalize(interactions)
norm_ratings.head()
del interactions

In [41]:
centered_rating_matrix = interactions_to_csc_matrix(norm_ratings, users_mapping=user_mapping, items_mapping=item_mapping, weight_col="norm_rating")

In [42]:
print("Compression ratio: ", 1 - centered_rating_matrix.getnnz() / (centered_rating_matrix.shape[0] * centered_rating_matrix.shape[1]))

Compression ratio:  0.9369533063577546


Итак, мы вычислили средний рейтинг для каждого пользователя. Теперь мы готовы к вычислению похожестей айтемов.

In [43]:
def adjusted_cosine_sim_gpu(csc_matrix: sparse.csc_matrix, device: cupy.cuda.Device, min_estimation: int):
    csc_matrix_gpu = cusparse.csc_matrix(csc_matrix)
    num_items = csc_matrix_gpu.shape[1]
    grid_size, thread_size = get_grid_thread_size(device, num_items)
    cuda_dtype = cupy.float64
    distances = cupy.zeros((num_items, num_items), dtype=cuda_dtype)  
    ADJUSTED_COSINE_SIM_BETWEEN_ITEMS(grid_size, thread_size, 
                                  (cupy.uint(num_items), 
                                   cupy.uint(min_estimation),
                                   csc_matrix_gpu.indptr.astype(cupy.uint), 
                                   csc_matrix_gpu.indices.astype(cupy.uint), 
                                   csc_matrix_gpu.data.astype(cuda_dtype), 
                                   distances))
    
    return cupy.asnumpy(distances)

In [44]:
with cupy.cuda.Device(0) as device:
    rid_size, thread_size = get_grid_thread_size(device, centered_rating_matrix.shape[1])
    distances = adjusted_cosine_sim_gpu(centered_rating_matrix, device, min_estimation=0)

In [45]:
# some difference because explicit numeric computing on the GPU with custom kernel may be unstable
assert math.isclose(distances[0, 0], 1.0)
assert math.isclose(distances[1, 2], 0.1069226, abs_tol=1e-7)
assert math.isclose(distances[1, 3], 0.0555092, abs_tol=1e-7)
assert math.isclose(distances[1, 5], -0.125509, abs_tol=1e-6)
assert math.isclose(distances[1, 1431], 1.0)
assert math.isclose(distances[4, 1123], 0.0)

In [46]:
def sort_simil_with_neighs(similarities: np.ndarray):
    sorted_sim = similarities.copy()
    
    np.fill_diagonal(sorted_sim, similarities.min() - 1)
    neighbors = np.flip(np.argsort(sorted_sim, axis=1), axis=1)

    for i in range(len(sorted_sim)):
        sorted_sim[i] = sorted_sim[i][neighbors[i]]

    assert neighbors.shape == sorted_sim.shape

    return sorted_sim, neighbors

In [47]:
@numba.jit(signature_or_function=numba.void(
        numba.uint32, 
        numba.uint32, 
        numba.uint32[:], 
        numba.uint32[:],
        numba.double[:],
        numba.double[:, :]
        ),
        nopython=True,
        parallel=True)
def cosine_sim_cpu(
    num_items,
    min_estimations,
    csc_col_pointers,
    csc_row_indices,
    csc_values,
    distance_matrix):
    for item_i in numba.prange(num_items):
        for item_j in numba.prange(item_i, num_items):
            
            if item_i == item_j:
                distance_matrix[item_i, item_j] = 1.0
                distance_matrix[item_j, item_i] = distance_matrix[item_i, item_j]
                continue

            start_i = csc_col_pointers[item_i]
            end_i = csc_col_pointers[item_i + 1]

            start_j = csc_col_pointers[item_j]
            end_j = csc_col_pointers[item_j + 1]

            num_estimation = 0

            while start_i < end_i and start_j < end_j:
                if csc_row_indices[start_i] == csc_row_indices[start_j]:
                    num_estimation += 1
                    start_i += 1
                    start_j += 1
                elif csc_row_indices[start_i] < csc_row_indices[start_j]:
                    start_i += 1
                else:
                    start_j += 1

            if num_estimation < min_estimations:
                distance_matrix[item_i, item_j] = 0.0
                distance_matrix[item_j, item_i] = distance_matrix[item_i, item_j]
                continue

            start_i = csc_col_pointers[item_i]
            start_j = csc_col_pointers[item_j]

            num = 0.0
            denum_i = 0.0
            denum_j = 0.0

            while start_i < end_i and start_j < end_j:
                if csc_row_indices[start_i] == csc_row_indices[start_j]:
                    num += csc_values[start_i] * csc_values[start_j]
                    denum_i += csc_values[start_i] ** 2
                    denum_j += csc_values[start_j] ** 2
                    start_i += 1
                    start_j += 1
                elif csc_row_indices[start_i] < csc_row_indices[start_j]:
                    start_i += 1
                else:
                    start_j += 1

            if np.isclose(denum_i, 0.0):
                inv_denum_i = 0.0
            else:
                inv_denum_i = 1 / np.sqrt(denum_i)
            
            if np.isclose(denum_j, 0.0):
                inv_denum_j = 0.0
            else:
                inv_denum_j = 1 / np.sqrt(denum_j)

            distance_matrix[item_i, item_j] = num * inv_denum_j * inv_denum_i
            distance_matrix[item_j, item_i] = distance_matrix[item_i, item_j]

In [48]:
distance_cpu = np.zeros((centered_rating_matrix.shape[1], centered_rating_matrix.shape[1]), dtype=np.float64)

In [49]:
cosine_sim_cpu(
    centered_rating_matrix.shape[1],
    0,
    centered_rating_matrix.indptr.astype(np.uint32),
    centered_rating_matrix.indices.astype(np.uint32),
    centered_rating_matrix.data.astype(np.float64),
    distance_cpu
)

In [50]:
distance_cpu[0, 1475]

-1.0

In [51]:
np.allclose(distances, distance_cpu)

True

In [52]:
sorted_sim, neighbors = sort_simil_with_neighs(distances)

In [53]:
distances[0, 1475]

-1.0

Посмотрим глазами на списки соседей, которые мы получили.

In [54]:
def show_neighs(item: pl.DataFrame, similarities: np.ndarray, neighbors: np.ndarray, inv_item_mapping: dict, k: int = 5):
    sim = similarities[:, :k].reshape(-1)
    neighs = neighbors[:, :k].reshape(-1)

    most_sim_items = pl.DataFrame(
        {
            "item_id": np.arange(len(similarities)).repeat(k),
            "sim": sim,
            "sim_item_id": neighs
        },
        schema={"item_id": item.schema["item_id"], "sim_item_id": item.schema["item_id"], "sim": pl.Float32}
    ).with_columns(
        pl.col("sim_item_id").apply(inv_item_mapping.get).alias("sim_item_id").cast(item.schema["item_id"]),
        pl.col("item_id").apply(inv_item_mapping.get).alias("item_id").cast(item.schema["item_id"])
    )

    return most_sim_items.join(item, left_on="sim_item_id", right_on="item_id", how="inner").join(
        item.select(pl.col("item_id"), pl.col("title").alias("orig_title")), on="item_id", how="inner")

In [55]:
most_sim_items = show_neighs(items, sorted_sim, neighbors, inv_item_mapping, 5)

In [56]:
with pl.Config() as cfg:
    cfg.set_fmt_str_lengths(200)
    print(most_sim_items.head())
    print(most_sim_items.tail())

shape: (5, 5)
┌─────────┬─────────────┬─────┬────────────────────────────────┬──────────────────┐
│ item_id ┆ sim_item_id ┆ sim ┆ title                          ┆ orig_title       │
│ ---     ┆ ---         ┆ --- ┆ ---                            ┆ ---              │
│ u32     ┆ u32         ┆ f64 ┆ str                            ┆ str              │
╞═════════╪═════════════╪═════╪════════════════════════════════╪══════════════════╡
│ 1       ┆ 1657        ┆ 1.0 ┆ Target (1995)                  ┆ Toy Story (1995) │
│ 1       ┆ 1450        ┆ 1.0 ┆ Golden Earrings (1947)         ┆ Toy Story (1995) │
│ 1       ┆ 1261        ┆ 1.0 ┆ Run of the Country, The (1995) ┆ Toy Story (1995) │
│ 1       ┆ 1493        ┆ 1.0 ┆ Modern Affair, A (1995)        ┆ Toy Story (1995) │
│ 1       ┆ 1627        ┆ 1.0 ┆ Wife, The (1995)               ┆ Toy Story (1995) │
└─────────┴─────────────┴─────┴────────────────────────────────┴──────────────────┘
shape: (5, 5)
┌─────────┬─────────────┬─────┬─────────────────

Как вам кажется, получились ли у нас хорошие результаты? Что объединяет нерелевантные айтемы из топов по похожестям?

In [57]:
with cupy.cuda.Device(0) as device:
    rid_size, thread_size = get_grid_thread_size(device, centered_rating_matrix.shape[1])
    distances = adjusted_cosine_sim_gpu(centered_rating_matrix, device, min_estimation=20)

In [58]:
assert np.isclose(distances[1, 1431], 0)
assert np.isclose(distances[1, 17], 0)
assert np.isclose(distances[4, 1123], 0)
assert np.isclose(distances[914, 1681], 0)

In [59]:
sorted_sim, neighbors = sort_simil_with_neighs(distances)

Посмотрим, что получилось теперь:

In [60]:
most_sim_items = show_neighs(items, sorted_sim, neighbors, inv_item_mapping, 5)

In [61]:
with pl.Config() as cfg:
    cfg.set_fmt_str_lengths(200)
    print(most_sim_items.head())
    print(most_sim_items.tail())

shape: (5, 5)
┌─────────┬─────────────┬──────────┬──────────────────────────────┬──────────────────┐
│ item_id ┆ sim_item_id ┆ sim      ┆ title                        ┆ orig_title       │
│ ---     ┆ ---         ┆ ---      ┆ ---                          ┆ ---              │
│ u32     ┆ u32         ┆ f64      ┆ str                          ┆ str              │
╞═════════╪═════════════╪══════════╪══════════════════════════════╪══════════════════╡
│ 1       ┆ 921         ┆ 0.538191 ┆ Farewell My Concubine (1993) ┆ Toy Story (1995) │
│ 1       ┆ 500         ┆ 0.511432 ┆ Fly Away Home (1996)         ┆ Toy Story (1995) │
│ 1       ┆ 923         ┆ 0.506015 ┆ Raise the Red Lantern (1991) ┆ Toy Story (1995) │
│ 1       ┆ 489         ┆ 0.461624 ┆ Notorious (1946)             ┆ Toy Story (1995) │
│ 1       ┆ 1152        ┆ 0.453082 ┆ In Love and War (1996)       ┆ Toy Story (1995) │
└─────────┴─────────────┴──────────┴──────────────────────────────┴──────────────────┘
shape: (5, 5)
┌─────────┬────

### Шаг 2. Выбрать топ рекомендаций для пользователя

Теперь, когда для каждого айтема есть список похожих, научимся формировать рекомендацию для пользователя.

#### Отбор кандидатов

1. Для каждого айтема из истории соберём сет из k самых похожих айтемов
2. Объединим полученные сеты
3. Уберём из полученного множества те айтемы, с которыми пользователь уже взаимодействовал

In [62]:
def candidate_items(interactions: pl.DataFrame, 
                    user_id: int, 
                    k: int, 
                    sorted_sim: np.ndarray, 
                    sorted_neighbors: np.ndarray, 
                    item_mapping: dict, 
                    inv_item_mapping: dict):
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    userid: id пользователя, для которого генерируются кандидаты
    k: количество кандидатов с каждого айтема из истории
    
    Возвращает 
        1. массив user_item_ids с id фильмов, просмотренных пользователем
        2. массив айтемов, близких к айтемам из истории пользователя
    """

    item_ids_hist = interactions.lazy().filter(pl.col("user_id") == user_id).select(pl.col("item_id").unique()).collect().to_series()
    zero_item_indices = item_ids_hist.apply(item_mapping.get)

    uniq_item_ids = np.unique(sorted_neighbors[zero_item_indices, :k].reshape(-1))
    
    return item_ids_hist, np.array(tuple(map(inv_item_mapping.get, np.setdiff1d(uniq_item_ids, item_ids_hist))))

In [63]:
user_item_ids, u_candidates = candidate_items(norm_ratings, 1, sorted_sim.shape[1], sorted_sim, neighbors, item_mapping, inv_item_mapping)

print('Количество просмотренных фильмов пользователя 1:', len(user_item_ids))
print('Количество кандидатов для пользователя 1:', len(u_candidates))

Количество просмотренных фильмов пользователя 1: 272
Количество кандидатов для пользователя 1: 1410


In [64]:
user_item_ids_test, u_candidates_test = candidate_items(norm_ratings, 1, sorted_sim.shape[1], sorted_sim, neighbors, item_mapping, inv_item_mapping)
assert len(user_item_ids_test) == 272
assert len(u_candidates_test) == 1410

user_item_ids_test, u_candidates_test = candidate_items(norm_ratings, 50, sorted_sim.shape[1], sorted_sim, neighbors, item_mapping, inv_item_mapping)

assert len(user_item_ids_test) == 24
assert len(u_candidates_test) == 1658

user_item_ids_test, u_candidates_test = candidate_items(norm_ratings, 942, sorted_sim.shape[1], sorted_sim, neighbors, item_mapping, inv_item_mapping)
assert len(user_item_ids_test) == 79
assert len(u_candidates_test) == 1603

del user_item_ids_test, u_candidates_test

#### Вычисление похожести между кандидатом и множеством айтемов из истории пользователя u

In [65]:
def similarity_with_user_items(
                            interactions: pl.DataFrame,
                            user_id: int,
                            item_id: int, 
                            user_item_ids: np.ndarray,
                            distance_matrix: np.ndarray, 
                            item_mapping: dict) -> float:
    """
    item_id: id айтема-кандидата, для которого считается похожесть с историей пользователя
    user_item_ids: массив id фильмов, просмотренных пользователем
    similarities: массив похожестей айтемов
    neighbors: массив соседей для всех айтемов
    
    Возвращает число – похожесть айтема на историю пользователя.
    """

    similiarities = distance_matrix[item_mapping[item_id], list(map(item_mapping.get, user_item_ids))]

    items_info = pl.DataFrame(
        {
            "item_id": user_item_ids,
        },
        schema={"item_id": interactions.schema["item_id"]}
    ).lazy().join(
        interactions.filter((pl.col("user_id") == user_id)).lazy(), on="item_id", how="left"
    ).select(
        pl.col("norm_rating"),
        pl.col("mean_user_rating")
    ).collect()

    mean_user_rating = items_info.select(pl.col("mean_user_rating").max())[0, 0]
    centered_rating = items_info.select(pl.col("norm_rating")).to_series().to_numpy()

    norm = np.linalg.norm(similiarities, ord=1)

    if math.isclose(norm, 0):
        inv_norm = 0.0
    else:
        inv_norm = 1.0 / norm

    return mean_user_rating + centered_rating.dot(similiarities) * inv_norm

#### Ранжирование кандидатов по их похожестям на историю пользователя

In [66]:
def rank_candidates(interactions: pl.DataFrame, user_id: int, candidates: np.ndarray, user_item_ids: np.ndarray, distance_matrix: np.ndarray, item_mapping: dict):
    """
    candidates: массив id фильмов-кандидатов
    user_item_ids: массив id фильмов, просмотренных пользователем
    similarities: массив похожестей айтемов
    neighbors: массив соседей для всех айтемов
    
    Возвращает массив tuple, где первый элемент – айди айтема, второй – похожесть на историю пользователя
    """
    
    # list of candidate items mapped to their corresponding similarities to user_item_ids
    sims = [similarity_with_user_items(interactions, user_id, c, user_item_ids, distance_matrix, item_mapping) for c in candidates]
    mapping = list(zip(candidates, sims))
    ranked_candidates = sorted(mapping, key=lambda couple: couple[1], reverse=True)

    return np.array(ranked_candidates)

In [67]:
user_id = 1

user_item_ids_test, candidates_test = candidate_items(norm_ratings, user_id, sorted_sim.shape[1], sorted_sim, neighbors, item_mapping, inv_item_mapping)

In [68]:
assert len(rank_candidates(norm_ratings, user_id, candidates_test, user_item_ids_test, distances, item_mapping)) == len(candidates_test)

## Соберём всё вместе

Теперь у нас есть всё, что нужно: отбор кандидатов, ранжирующая функция и мы готовы собрать весь пайплайн item-to-item рекомендаций.

In [69]:
def topn_recommendation(norm_ratings: pl.DataFrame, items: pl.DataFrame, user_id: int, distance_matrix: np.ndarray, sorted_sim: np.ndarray, neighbors: np.ndarray, item_mapping: dict, k=None, N=30):
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    userid: id пользователя, для которого генерируются кандидаты
    similarities: массив похожестей айтемов
    neighbors: массив соседей для всех айтемов
    k: количество кандидатов на стадии отбора кандидатов
    N: количество рекомендаций фильмов для пользователя
    
    Возвращает dataframe c рекомендацией top-N фильмов для пользователя userid.
    """
    if k is None:
        k = sorted_sim.shape[1]

    # find candidate items
    user_item_ids, candidates = candidate_items(norm_ratings, user_id, k, sorted_sim, neighbors, item_mapping, inv_item_mapping)
    
    # rank candidate items according to their similarities with user_item_ids
    ranked_candidates = rank_candidates(norm_ratings, user_id, candidates, user_item_ids, distance_matrix, item_mapping)
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    top_n = pl.from_records(ranked_candidates[:N], schema={"item_id": norm_ratings.schema["item_id"], "sim": pl.Float32})

    return top_n.join(items.select(pl.col("item_id"), pl.col("title")), on="item_id", how="inner")

Посмотрим, как это работает:

In [70]:
rec = topn_recommendation(norm_ratings, items, 1, distances, sorted_sim, neighbors, item_mapping)

In [71]:
with pl.Config() as config:
    config.set_fmt_str_lengths(200)
    print(rec.head(n=10))

shape: (10, 3)
┌─────────┬──────────┬─────────────────────────────────────────────────────────────────────────────┐
│ item_id ┆ sim      ┆ title                                                                       │
│ ---     ┆ ---      ┆ ---                                                                         │
│ u32     ┆ f32      ┆ str                                                                         │
╞═════════╪══════════╪═════════════════════════════════════════════════════════════════════════════╡
│ 1       ┆ 4.348933 ┆ Toy Story (1995)                                                            │
│ 297     ┆ 4.400891 ┆ Ulee's Gold (1997)                                                          │
│ 318     ┆ 4.380766 ┆ Schindler's List (1993)                                                     │
│ 357     ┆ 4.377169 ┆ One Flew Over the Cuckoo's Nest (1975)                                      │
│ …       ┆ …        ┆ …                                                    

А теперь попробуем применить то, что у нас получилось на тестовом пользователе, для которого соберём историю просмотров сами.

In [72]:
user_id = 2
test_hist = norm_ratings.filter(pl.col("user_id") == user_id).sample(fraction=0.1)

In [73]:
rec = topn_recommendation(test_hist, items, user_id, distances, sorted_sim, neighbors, item_mapping)

In [74]:
with pl.Config() as config:
    config.set_fmt_str_lengths(200)
    
    print(test_hist.join(items, on="item_id", how="inner"), rec.head(n=10))

shape: (6, 6)
┌─────────┬─────────┬────────┬──────────────────┬─────────────┬────────────────────────────────┐
│ user_id ┆ item_id ┆ rating ┆ mean_user_rating ┆ norm_rating ┆ title                          │
│ ---     ┆ ---     ┆ ---    ┆ ---              ┆ ---         ┆ ---                            │
│ u32     ┆ u32     ┆ f32    ┆ f32              ┆ f32         ┆ str                            │
╞═════════╪═════════╪════════╪══════════════════╪═════════════╪════════════════════════════════╡
│ 2       ┆ 25      ┆ 4.0    ┆ 3.709677         ┆ 0.290323    ┆ Birdcage, The (1996)           │
│ 2       ┆ 111     ┆ 4.0    ┆ 3.709677         ┆ 0.290323    ┆ Truth About Cats & Dogs, The   │
│         ┆         ┆        ┆                  ┆             ┆ (1996)                         │
│ 2       ┆ 272     ┆ 5.0    ┆ 3.709677         ┆ 1.290323    ┆ Good Will Hunting (1997)       │
│ 2       ┆ 284     ┆ 4.0    ┆ 3.709677         ┆ 0.290323    ┆ Tin Cup (1996)                 │
│ 2       ┆ 307 

Итак, в этом семинаре мы научились строить item-to-item рекомендации. Этот подход можно улучшать и развивать. 

1. Например, мы можем учитывать рейтинги айтемов из истории пользователя. Определить, когда рейтинг был позитивный и учитывать кандидатов только для таких айтемов
2. Можем поэксеприментировать над определением похожести